In [1]:
from conftest import setup_project
setup_project()

env: CUDA_VISIBLE_DEVICES=0


In [2]:
import jax
import jax.numpy as jnp

from JAxtar.hash import hash_func_builder
from puzzle.lightsout import LightsOut
from heuristic.lightsout_heuristic import LightsOutHeuristic

In [3]:
puzzle = LightsOut(7)
heuristic = LightsOutHeuristic(puzzle)

2024-09-25 07:59:58.578908: W external/xla/xla/service/gpu/nvptx_compiler.cc:765] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.5.82). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [4]:
#check batch generation
states = puzzle.get_target_state()
print(states)
next_states, costs = puzzle.get_neighbours(states)
print(next_states)
print(costs)

┏━━━━━━━━━━━━━━━┓
┃ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ ┃
┗━━━━━━━━━━━━━━━┛
┏━━━━━━━━━━━━━━━┓  ┏━━━━━━━━━━━━━━━┓  ┏━━━━━━━━━━━━━━━┓  ...              ┏━━━━━━━━━━━━━━━┓  ┏━━━━━━━━━━━━━━━┓  ┏━━━━━━━━━━━━━━━┓
┃ ■ ■ □ □ □ □ □ ┃  ┃ ■ □ □ □ □ □ □ ┃  ┃ □ □ □ □ □ □ □ ┃  (batch : (49,))  ┃ □ □ □ □ □ □ □ ┃  ┃ □ □ □ □ □ □ □ ┃  ┃ □ □ □ □ □ □ □ ┃
┃ ■ □ □ □ □ □ □ ┃  ┃ ■ ■ □ □ □ □ □ ┃  ┃ ■ □ □ □ □ □ □ ┃                   ┃ □ □ □ □ □ □ □ ┃  ┃ □ □ □ □ □ □ □ ┃  ┃ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ ┃  ┃ ■ □ □ □ □ □ □ ┃  ┃ ■ ■ □ □ □ □ □ ┃                   ┃ □ □ □ □ □ □ □ ┃  ┃ □ □ □ □ □ □ □ ┃  ┃ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ ┃  ┃ □ □ □ □ □ □ □ ┃  ┃ ■ □ □ □ □ □ □ ┃                   ┃ □ □ □ □ □ □ ■ ┃  ┃ □ □ □ □ □ □ □ ┃  ┃ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ ┃  ┃ □ □ □ □ □ □ □ ┃  ┃ □ □ □ □ □ □ □ ┃                   ┃ □ □ □ □ □ ■ ■ ┃  ┃ □ □ □ □ □ □ ■ ┃  ┃ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ ┃  ┃ □ □ □ □ □ □ □ ┃  ┃ □ □ □ □ □ □ □ ┃   

In [5]:
print(puzzle.is_solved(states, states))
print(puzzle.is_solved(states, next_states[0]))

True
False


In [6]:
# check batch neighbours
states = jax.vmap(puzzle.get_initial_state, in_axes=0)(key=jax.random.split(jax.random.PRNGKey(0),int(1e3))) # total 10 million states
next_states, costs = jax.vmap(puzzle.get_neighbours, in_axes=0)(states)
print(next_states)
next_states = next_states.flatten()
costs = costs.flatten()
print(next_states.shape)
print(next_states.batch_shape)
print(next_states.dtype)
print(costs.shape)

┏━━━━━━━━━━━━━━━┓  ┏━━━━━━━━━━━━━━━┓  ┏━━━━━━━━━━━━━━━┓  ...                   ┏━━━━━━━━━━━━━━━┓  ┏━━━━━━━━━━━━━━━┓  ┏━━━━━━━━━━━━━━━┓
┃ ■ ■ □ □ □ ■ ■ ┃  ┃ ■ □ □ □ □ ■ ■ ┃  ┃ □ □ □ □ □ ■ ■ ┃  (batch : (1000, 49))  ┃ □ □ □ ■ ■ ■ ■ ┃  ┃ □ □ □ ■ ■ ■ ■ ┃  ┃ □ □ □ ■ ■ ■ ■ ┃
┃ ■ □ □ □ □ □ ■ ┃  ┃ ■ ■ □ □ □ □ ■ ┃  ┃ ■ □ □ □ □ □ ■ ┃                        ┃ □ □ ■ □ ■ ■ □ ┃  ┃ □ □ ■ □ ■ ■ □ ┃  ┃ □ □ ■ □ ■ ■ □ ┃
┃ □ □ ■ □ □ □ □ ┃  ┃ ■ □ ■ □ □ □ □ ┃  ┃ ■ ■ ■ □ □ □ □ ┃                        ┃ □ ■ ■ ■ □ ■ □ ┃  ┃ □ ■ ■ ■ □ ■ □ ┃  ┃ □ ■ ■ ■ □ ■ □ ┃
┃ □ ■ □ ■ □ □ ■ ┃  ┃ □ ■ □ ■ □ □ ■ ┃  ┃ ■ ■ □ ■ □ □ ■ ┃                        ┃ □ □ ■ ■ □ □ ■ ┃  ┃ □ □ ■ ■ □ □ □ ┃  ┃ □ □ ■ ■ □ □ □ ┃
┃ □ □ □ ■ □ ■ ■ ┃  ┃ □ □ □ ■ □ ■ ■ ┃  ┃ □ □ □ ■ □ ■ ■ ┃                        ┃ □ ■ ■ ■ ■ □ ■ ┃  ┃ □ ■ ■ ■ ■ ■ ■ ┃  ┃ □ ■ ■ ■ ■ ■ □ ┃
┃ □ ■ □ □ □ □ ■ ┃  ┃ □ ■ □ □ □ □ ■ ┃  ┃ □ ■ □ □ □ □ ■ ┃                        ┃ ■ ■ ■ ■ □ ■ ■ ┃  ┃ ■ ■ ■ ■ □ □ ■ ┃  ┃ ■ ■ ■ ■ □ ■ ■ ┃
┃ ■ □ □ □ □ □ □ ┃  ┃ ■ □ □ □ □ □ □ ┃  ┃ ■ □ □ □ □ □ □ ┃

In [7]:
puzzle_hash_fun: callable = hash_func_builder(puzzle.State)

In [8]:
#check hashing
hashes = jax.vmap(puzzle_hash_fun, in_axes=(0, None))(states, 1)
#count hash collision
print(hashes.shape)
print(hashes.dtype)
print(jnp.unique(hashes).shape) # Low collision
print(jnp.unique(states.board, axis=0).shape) # Low collision

hashes = jax.vmap(puzzle_hash_fun, in_axes=(0, None))(next_states, 1)
#count hash collision
print(hashes.shape)
print(hashes.dtype)
print(jnp.unique(hashes).shape) # High collision
print(jnp.unique(next_states.board, axis=0).shape) # High collision

(1000,)
uint32
(1000,)
(1000, 7)
(49000,)
uint32
(48978,)
(48978, 7)


In [9]:
#check heuristic
print("Heuristic")
dist = jax.vmap(heuristic.distance, in_axes=(0, None))(next_states, states[0])
print(dist)

Heuristic
[0.6 0.8 0.8 ... 4.4 4.  4.6]


In [10]:
from heuristic.DAVI.neuralheuristic.lightsout_neuralheuristic import LightsOutNeuralHeuristic
neural_heuristic = LightsOutNeuralHeuristic.load_model(puzzle, "../heuristic/DAVI/neuralheuristic/params/lightsout_7.pkl")
neural_heuristic_fn = neural_heuristic.distance
target_state = puzzle.get_target_state()
target_neighbours, costs = puzzle.get_neighbours(target_state)
print(target_neighbours)
dist = jax.vmap(neural_heuristic_fn, in_axes=(0, None))(target_neighbours.flatten(), target_state)
print(dist)
print(dist.shape)
print(dist.dtype)
print(jnp.mean(dist))

dist = jax.vmap(neural_heuristic_fn, in_axes=(0, None))(next_states[:100], target_state)
print(dist)
print(dist.shape)
print(dist.dtype)
print(jnp.mean(dist))

Could not load symbol cuFuncGetName. Error: /usr/lib/x86_64-linux-gnu/libcuda.so.1: undefined symbol: cuFuncGetName


┏━━━━━━━━━━━━━━━┓  ┏━━━━━━━━━━━━━━━┓  ┏━━━━━━━━━━━━━━━┓  ...              ┏━━━━━━━━━━━━━━━┓  ┏━━━━━━━━━━━━━━━┓  ┏━━━━━━━━━━━━━━━┓
┃ ■ ■ □ □ □ □ □ ┃  ┃ ■ □ □ □ □ □ □ ┃  ┃ □ □ □ □ □ □ □ ┃  (batch : (49,))  ┃ □ □ □ □ □ □ □ ┃  ┃ □ □ □ □ □ □ □ ┃  ┃ □ □ □ □ □ □ □ ┃
┃ ■ □ □ □ □ □ □ ┃  ┃ ■ ■ □ □ □ □ □ ┃  ┃ ■ □ □ □ □ □ □ ┃                   ┃ □ □ □ □ □ □ □ ┃  ┃ □ □ □ □ □ □ □ ┃  ┃ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ ┃  ┃ ■ □ □ □ □ □ □ ┃  ┃ ■ ■ □ □ □ □ □ ┃                   ┃ □ □ □ □ □ □ □ ┃  ┃ □ □ □ □ □ □ □ ┃  ┃ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ ┃  ┃ □ □ □ □ □ □ □ ┃  ┃ ■ □ □ □ □ □ □ ┃                   ┃ □ □ □ □ □ □ ■ ┃  ┃ □ □ □ □ □ □ □ ┃  ┃ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ ┃  ┃ □ □ □ □ □ □ □ ┃  ┃ □ □ □ □ □ □ □ ┃                   ┃ □ □ □ □ □ ■ ■ ┃  ┃ □ □ □ □ □ □ ■ ┃  ┃ □ □ □ □ □ □ □ ┃
┃ □ □ □ □ □ □ □ ┃  ┃ □ □ □ □ □ □ □ ┃  ┃ □ □ □ □ □ □ □ ┃                   ┃ □ □ □ □ □ □ ■ ┃  ┃ □ □ □ □ □ ■ ■ ┃  ┃ □ □ □ □ □ □ ■ ┃
┃ □ □ □ □ □ □ □ ┃  ┃ □ □ □ □ □ □ □ ┃  ┃ □ □ □ □ □ □ □ ┃                   ┃ □ □ □ □ □ □ □ 